In [67]:
#load and install required packages

In [68]:
#install required packages
!pip install sklearn_crfsuite
!pip install scikit-learn 
!pip install nltk 


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [69]:
# import required libraries
import pandas as pd 
import nltk
import sklearn
import scipy.stats
import numpy as np
import seaborn as sns
import sklearn_crfsuite
import csv

#from matplotlib 
from matplotlib import pyplot as plt

#from itertools
from itertools import chain

#from sklearn
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [70]:
#load the data and ontologies

In [71]:
#this function turns the file into a list. 
def file2list(fileLocation):
    outputList = []
    with open(fileLocation, 'r', encoding='utf8') as myfile:
        sentences = myfile.read().split('\n\n')
        for sentence in sentences:
                sentenceList = []
                words = sentence.split('\n')
                for word in words:
                    wordsList = []
                    attributes = word.split(' ')
                    for attribute in attributes:
                        wordsList.append(attribute)
                    sentenceList.append(wordsList)
                outputList.append(sentenceList)
    
    return outputList

In [72]:
# train =  file2list('D:\\phd-data\\NER-annotation-data\\Dutch\\5-folds-with-pos-with-cut-sentences\\fold1.txt') 
# test =  file2list('D:\\phd-data\\NER-annotation-data\\Dutch\\5-folds-with-pos-with-cut-sentences\\fold2.txt')

#load the datasets, with training documents as train, and test documents and test
fold = "fold5"
train =  file2list('data/5-folds-test-train-split/'+ fold +'/train.txt') 
test =  file2list('data/5-folds-test-train-split/'+ fold +'/test.txt')

#remove empty line at the end of the file as this breaks the code
test.pop() 
train.pop()

[['74', 'SPEC(symb)', '[74]', 'O']]

In [73]:
%%time 
# calculates the time to open the file
        #train the NER on the list. there is one set of test and one of training. often 20:80 split
train_sent = train
test_sent = test   # tests the sent (input) of the given list as defined above
train_sent[0] # displayes the first 10 rows in the bio. - each row hs the token (effectively word), followed by pos?, and the bio label
# to identify whats below - token - label(specific label) - common derivitive of word (for posting would be post)  - then the bio label

CPU times: total: 62.5 ms
Wall time: 48 ms


[['Rapportage', 'N(soort,ev,basis,zijd,stan)', '[rapport][age]', 'O'],
 ['Archeologische', 'ADJ(prenom,basis,met-e,stan)', '[archeologisch][e]', 'O'],
 ['Monumentenzorg',
  'N(soort,ev,basis,zijd,stan)',
  '[monument][en][zorg]',
  'O']]

In [74]:
#time to load the ontologies 

# Function to convert a csv file to a list of dictionaries.  Takes in one variable called "variables_file"
def getTermsFromCsv(variables_file):
    # Open variable-based csv, iterate over the rows and map values to a list of dictionaries containing key/value pairs
    reader = csv.DictReader(open(variables_file, 'r', encoding="utf-8"))
    output = []
    for line in reader:
        for columnName in line:
            if (columnName[0:7] == 'concept' or columnName[0:8] == 'altLabel') and len(line[columnName]) > 0:
                output.append(line[columnName].lower())
    return output

# Material ontology    
materials_list = getTermsFromCsv('ontologies/ABR-materialen-cleaned.csv')
print(materials_list[0:100]) # outputs the list of terms to see what sort of data it contains
print("\n""\n")


#Material ontology
# materials = pd.read_csv(r'ontologies\ABR-materialen-cleaned.csv') #open the file
# materials[materials.columns[2]]= materials[materials.columns[2]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
# materials_list = materials[materials.columns[2]].values.tolist() #turns the items into a list
# print(materials_list) # outputs the list of terms to see what sort of data it contains
# print("\n""\n")

#Taxon ontology
# taxon = pd.read_csv(r'ontologies\Taxon.tsv', sep="\t", error_bad_lines=False) #open the file
# taxon[taxon.columns[8]]= taxon[taxon.columns[8]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
# taxon_list = taxon[taxon.columns[8]].values.tolist() #turns the items into a list
# print(taxon_list[0:100]) # outputs the list of terms to see what sort of data it contains
# print("\n""\n")

#Animal ontology ### NOT IN word2features!
# animal = pd.read_csv(r'ontologies\VernacularName.tsv', sep="\t") #open the file
# animal[animal.columns[2]]= animal[animal.columns[2]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
# animal_list = animal[animal.columns[2]].values.tolist() #turns the items into a list
# print(animal_list[0:100]) # outputs the list of terms to see what sort of data it contains
# print("\n""\n")

# Artefact ontology    
artefact_list = getTermsFromCsv('ontologies/ABR-artefacten-cleaned.csv')
print(artefact_list[0:100]) # outputs the list of terms to see what sort of data it contains
print("\n""\n")







#artefact ontology
# artefact = pd.read_csv(r'ontologies\ABR-artefacten-cleaned.csv', sep="\t") #open the file
# artefact[artefact.columns[2]]= artefact[artefact.columns[2]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
# artefact_list = artefact[artefact.columns[2]].values.tolist() #turns the items into a list
# print(artefact_list[0:100]) # outputs the list of terms to see what sort of data it contains
# print("\n""\n")

#Periods ontology - for notes see above
periods = pd.read_csv(r'ontologies\periodo_extended.csv')
periods[periods.columns[1]]= periods[periods.columns[1]].str.lower()
periods_list = periods[periods.columns[1]].values.tolist()
print(periods_list[0:100])
print("\n""\n")


#Context ontology - for notes see material 
# context = pd.read_csv(r'ontologies\context.csv')
# context[context.columns[0]]= context[context.columns[0]].str.lower().str.replace('*','').str.replace('<','').str.replace('>','') #replace all other symbols
# context[context.columns[0]]= context[context.columns[0]].replace(to_replace =':.*',value='',regex=True) # replace the words after a colon
# context_list = context[context.columns[0]].values.tolist()
# print(context_list[0:100])
# print("\n""\n")

['steen', 'mortel', 'vuursteen', 'valkenburg', 'rijckholt', 'helgoland', 'natuursteen', 'jadeiet', 'helleflint', 'leisteen', 'grauwacke', 'tufsteen', 'marmer', 'gabbro', 'graniet', 'ijzeroer', 'basalt', 'kiezelgeode', 'kalksteen', 'mergel', 'oker', 'zandsteen', 'kwartsiet', 'dioriet', 'bergkristal', 'diabaas', 'kleischalie', 'kleisteen', 'porfier', 'amfiboliet', 'tefriet', 'lydiet', 'phtaniet', 'pleister', 'metaal', 'koper', 'brons', 'ijzer', 'gietijzer', 'staal', 'lood', 'goud', 'bladgoud', 'tin', 'zilver', 'messing', 'zink', 'aluminium', 'papier', 'hout/houtskool', 'hout', 'houtskool', 'macroresten', 'zaden', 'bot', 'gewei', 'dierlijk bot', 'tand', 'ivoor', 'keratine', 'hoef', 'hoorn', 'balein', 'haar', 'nagel', 'schildpad', 'riet', 'fossiel', 'git', 'ligniet', 'sapropeliet', 'barnsteen', 'textiel', 'touw', 'visresten', 'leer', 'schelp', 'inktvisschilden', 'glas', 'keramiek', 'huttenleem', 'magering', 'calciet', 'potgruis', 'chamotte', 'kwarts', 'mica', 'schelpmagering', 'klei', 'pij

In [75]:
#time to use POS

In [76]:
#this calls the function
def word2features(sent, i): 
    word = sent[i][0] #takes each token
    postag = sent[i][1] #this is the Part of Speach Tager
    
    #this tells if each token is in the ontology or not 
    if word in materials_list: 
        in_materials = True 
    else:
        in_materials = False
    
#     if word in taxon_list: 
#         in_taxon = True 
#     else:
#         in_taxon = False
   
    if word in artefact_list: 
        in_artefact = True 
    else:
        in_artefact = False
 
    if word in periods_list: 
        in_periods = True 
    else:
        in_periods = False

#     if word in context_list: 
#         in_context = True 
#     else:
#         in_context = False
    
    #time to give each token some information     
    features = { # these are all default. 
        'bias': 1.0, # bias is just 1. 
        'word.lower()': word.lower(), # tells if the token is lower case 
        'word[-3:]': word[-3:], # takes the last four letters - the suffix
        'Word.in_materials': in_materials, #is the token in the material ontology
#         'Word.in_taxon': in_taxon, #is the token in the material ontology
        'Word.in_artefact': in_artefact, #is the token in the periods ontology 
        'Word.in_periods': in_periods, #is the token in the periods ontology
#         'Word.in_context': in_context, #is the token in the evidence ontology
        'word.isupper()': word.isupper(), # tells if the whole token is uppercase 
        'word.istitle()': word.istitle(), # tells if the token is capital first letter
        'postag': postag,  # what is its label - Part-Of-Speech Tagger
        'postag[:2]': postag[:2],  #Takes the first three letters of the tag
        'word.isdigit()': word.isdigit(),
    }
    if i > 0: #if the token is not at the start of a sentence
        word1 = sent[i-1][0] # works out details of the token before - this is to understand the context 
        postag1 = sent[i-1][1] #what is the postag of the word before
        wordbefore = (sent[i-1][0]+ ' ' +sent[i][0]).lower #this is the token and the token before
        #if this word and word before is in the ontology then 
        if wordbefore in materials_list: 
            wordbefore_in_materials = True 
        else:
            wordbefore_in_materials = False

#         if wordbefore in taxon_list: 
#             wordbefore_in_taxon = True 
#         else:
#             wordbefore_in_taxon = False

        if wordbefore in artefact_list: 
            wordbefore_in_artefact = True 
            print(word)
        else:
            wordbefore_in_artefact = False

        if wordbefore in periods_list: 
            wordbefore_in_periods = True 
        else:
            wordbefore_in_periods = False

#         if wordbefore in context_list: 
#             wordbefore_in_context = True 
#         else:
#             wordbefore_in_context = False
        features.update({
            '-1:word.lower()': word1.lower(), # tells if the token is lower case
            '-1:word.istitle()': word1.istitle(), # tells if the token is capital first letter
            '-1:word.isdigit()': word1.isdigit(), # tells if the toekn is only numbers
            '-1:word.isupper()': word1.isupper(),# tells if the whole token is uppercase
            'wordbefore_in_materials': wordbefore_in_materials,
#             'wordbefore_in_taxon': wordbefore_in_taxon,
            'wordbefore_in_artefact': wordbefore_in_artefact,
            'wordbefore_in_periods': wordbefore_in_periods,
#             'wordbefore_in_context': wordbefore_in_context,
            '-1:postag': postag1, # what was its POS tag
            '-1:postag[:2]': postag1[:2], #what is the first three POS tag of the word before
        })
    
        if i > 1: #if the token is not at the start of a sentence
                word3 = sent[i-1][0] # works out details of the token before - this is to understand the context 
                postag3 = sent[i-1][1] #what is the postag of the word before
                two_words_before = (sent[i-1][0]+ ' ' +sent[i-1][0]+ ' ' +sent[i][0]).lower #this is the token and the token before
                #if this word and word before is in the ontology then 
                if two_words_before in materials_list: 
                    two_words_before_in_materials = True 
                else:
                    two_words_before_in_materials = False

#                 if two_words_before in taxon_list: 
#                     two_words_before_in_taxon = True 
#                 else:
#                     two_words_before_in_taxon = False

                if two_words_before in artefact_list: 
                    two_words_before_in_artefact = True 
                    print(word)
                else:
                    two_words_before_in_artefact = False

                if two_words_before in periods_list: 
                    two_words_before_in_periods = True 
                else:
                    two_words_before_in_periods = False

#                 if two_words_before in context_list: 
#                     two_words_before_in_context = True 
#                 else:
#                     two_words_before_in_context = False
                features.update({
                    '-1:word.lower()': word1.lower(), # tells if the token is lower case
                    '-1:word.istitle()': word1.istitle(), # tells if the token is capital first letter
                    '-1:word.isdigit()': word1.isdigit(), # tells if the toekn is only numbers
                    '-1:word.isupper()': word1.isupper(),# tells if the whole token is uppercase
                    'two_words_before_in_materials': two_words_before_in_materials,
#                     'two_words_before_in_taxon': two_words_before_in_taxon,
                    'two_words_before_in_artefact': two_words_before_in_artefact,
                    'two_words_before_in_periods': two_words_before_in_periods,
#                     'two_words_before_in_context': two_words_before_in_context,
                    '-2-postag': postag3, # what was its POS tag
                    '-2:postag[:2]': postag3[:2], #what is the first three POS tag of the word before
                })
        else:
            features['BOS2'] = True # if word is the beggining of sentence label it as so         
    else:
        features['BOS'] = True # if word is the beggining of sentence label it as so 
        
    if i < len(sent)-1: # is the word at the end of the sentence. sme as above after
        wordafter= (sent[i][0]+ ' ' +sent[i+1][0]).lower
        #this tells if the token AFTER and each token combined is in the ontology or not
        if wordafter in materials_list: 
            wordafter_in_materials = True 
        else:
            wordafter_in_materials = False

#         if wordafter in taxon_list: 
#             wordafter_in_taxon = True 
#         else:
#             wordafter_in_taxon = False

        if wordafter in artefact_list: 
            wordafter_in_artefact = True 
        else:
            wordafter_in_artefact = False

        if wordafter in periods_list: 
            wordafter_in_periods = True 
        else:
            wordafter_in_periods = False

#         if wordafter in context_list: 
#             wordafter_in_context = True 
#         else:
#             wordafter_in_context = False
        word2 = sent[i+1][0]  
        postag2 = sent[i+1][1] 
        features.update({
            'wordafter_in_materials': wordafter_in_materials,
#             'wordafter_in_taxon': wordafter_in_taxon,
            'wordafter_in_artefact': wordafter_in_artefact,
            'wordafter_in_periods': wordafter_in_periods,
#             'wordafter_in_context': wordafter_in_context,
            '+2:word.lower()': word2.lower(),
            '+2:word.istitle()': word2.istitle(),
            '+2:word.isupper()': word2.isupper(),
            '+2:postag': postag2,
            '+2:postag[:2]': postag2[:2],
        })
        
        if i < len(sent)-2: #if the token is not at the start of a sentence
                word4 = sent[i-1][0] # works out details of the token before - this is to understand the context 
                postag4 = sent[i-1][1] #what is the postag of the word before
                two_words_after = (sent[i][0]+ ' ' +sent[i+1][0]+ ' ' +sent[i+1][0]).lower#this is the token and the token before
                #if this word and word before is in the ontology then 
                if two_words_after in materials_list: 
                    two_words_after_in_materials = True 
                else:
                    two_words_after_in_materials = False

#                 if two_words_after in taxon_list: 
#                     two_words_after_in_taxon = True 
#                 else:
#                     two_words_after_in_taxon = False

                if two_words_after in artefact_list: 
                    two_words_after_in_artefact = True 
                    print(word)
                else:
                    two_words_after_in_artefact = False

                if two_words_after in periods_list: 
                    two_words_after_in_periods = True 
                else:
                   two_words_after_in_periods = False

#                 if two_words_after in context_list: 
#                     two_words_after_in_context = True 
#                 else:
#                     two_words_after_in_context = False
                features.update({
                    '-1:word.lower()': word4.lower(), # tells if the token is lower case
                    '-1:word.istitle()': word4.istitle(), # tells if the token is capital first letter
                    '-1:word.isdigit()': word4.isdigit(), # tells if the toekn is only numbers
                    '-1:word.isupper()': word4.isupper(),# tells if the whole token is uppercase
                    'two_words_after_in_materials': two_words_after_in_materials,
#                     'two_words_after_in_taxon': two_words_after_in_taxon,
                    'two_words_after_in_artefact': two_words_after_in_artefact,
                    'two_words_after_in_periods': two_words_after_in_periods,
#                     'two_words_after_in_context': two_words_after_in_context,
                    '-2-postag': postag4, # what was its POS tag
                    '-2:postag[:2]': postag4[:2], #what is the first three POS tag of the word before
                })
        else:
            features['EOS2'] = True # if word is the beggining of sentence label it as so       
        
        
        
    else:
        features['EOS'] = True # if word is the end of sentence label it as so         
   

    if i < len(sent)-1 and i > 0: # is the word at the end of the sentence. sme as above after
            word3 = sent[i+1][0]  
            postag3 = sent[i+1][1] 
            wordsorround = (sent[i-1][0]+ ' ' +sent[i][0]+ ' ' +sent[i+1][0]).lower
            if wordsorround in materials_list: 
                wordsorround_in_materials = True 
            else:
                wordsorround_in_materials = False

#             if wordsorround in taxon_list: 
#                 wordsorround_in_taxon = True 
#             else:
#                 wordsorround_in_taxon = False

            if wordsorround in artefact_list: 
                wordsorround_in_artefact = True 
            else:
                wordsorround_in_artefact = False

            if wordsorround in periods_list: 
                wordsorround_in_periods = True 
            else:
                wordsorround_in_periods = False

#             if wordsorround in context_list: 
#                 wordsorround_in_context = True 
#             else:
#                 wordsorround_in_context = False
            features.update({
                'wordsorround_in_materials': wordsorround_in_materials,
# #                 'wordsorround_in_taxon': wordsorround_in_taxon,
                'wordsorround_in_artefact': wordsorround_in_artefact,
                'wordsorround_in_periods': wordsorround_in_periods,
#                 'wordsorround_in_context': wordsorround_in_context,
            })
            if i < len(sent)-2 and i > 1: # is the word at the end of the sentence. sme as above after
                word5 = sent[i+1][0]  
                postag5 = sent[i+1][1] 
                twowordsorround = (sent[i-2][0]+ ' ' +sent[i-1][0]+ ' ' +sent[i][0]+ ' ' +sent[i+1][0]+ ' ' +sent[i+2][0]).lower
                if twowordsorround in materials_list: 
                    twowordsorround_in_materials = True 
                else:
                    twowordsorround_in_materials = False

# #                 if twowordsorround in taxon_list: 
# #                     twowordsorround_in_taxon = True 
# #                 else:
# #                     twowordsorround_in_taxon = False

                if twowordsorround in artefact_list: 
                    twowordsorround_in_artefact = True 
                else:
                    twowordsorround_in_artefact = False

                if twowordsorround in periods_list: 
                    twowordsorround_in_periods = True 
                else:
                    twowordsorround_in_periods = False

#                 if twowordsorround in context_list: 
#                     twowordsorround_in_context = True 
#                 else:
#                     twowordsorround_in_context = False
                features.update({
                    'twowordsorround_in_materials': twowordsorround_in_materials,
# #                     'twowordsorround_in_taxon': twowordsorround_in_taxon,
                    'twowordsorround_in_artefact': twowordsorround_in_artefact,
                    'twowordsorround_in_periods': twowordsorround_in_periods,
#                     'twowordsorround_in_context': twowordsorround_in_context,                   
                })
            else:
                    features['OWS'] = True # if word is the end of sentence label it as so 
    else:
            features['OWS'] = True # if word is the end of sentence label it as so 
            
    return features # output these details
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))] #output for each word

def sent2labels(sent):
    return [label for token, postag, POS, label in sent] #output for each token

def sent2tokens(sent):
    return [token for token, postag, POS, label in sent] #output for ???


In [77]:
#print(sent2features[:2])

In [78]:
#sent2features(train_sent[0:1])[0]
[sent2features(s) for s in train_sent[0:1]]

[[{'bias': 1.0,
   'word.lower()': 'rapportage',
   'word[-3:]': 'age',
   'Word.in_materials': False,
   'Word.in_artefact': False,
   'Word.in_periods': False,
   'word.isupper()': False,
   'word.istitle()': True,
   'postag': 'N(soort,ev,basis,zijd,stan)',
   'postag[:2]': 'N(',
   'word.isdigit()': False,
   'BOS': True,
   'wordafter_in_materials': False,
   'wordafter_in_artefact': False,
   'wordafter_in_periods': False,
   '+2:word.lower()': 'archeologische',
   '+2:word.istitle()': True,
   '+2:word.isupper()': False,
   '+2:postag': 'ADJ(prenom,basis,met-e,stan)',
   '+2:postag[:2]': 'AD',
   '-1:word.lower()': 'monumentenzorg',
   '-1:word.istitle()': True,
   '-1:word.isdigit()': False,
   '-1:word.isupper()': False,
   'two_words_after_in_materials': False,
   'two_words_after_in_artefact': False,
   'two_words_after_in_periods': False,
   '-2-postag': 'N(soort,ev,basis,zijd,stan)',
   '-2:postag[:2]': 'N(',
   'OWS': True},
  {'bias': 1.0,
   'word.lower()': 'archeologis

In [79]:
#time to train the NER

In [80]:
%%time
X_train = [sent2features(s) for s in train_sent] # for the token train the ner on the learned set
y_train = [sent2labels(s) for s in train_sent] # for the POS tag train the ner on the learned set

X_test = [sent2features(s) for s in test_sent] # for the token train the ner on the test set
y_test = [sent2labels(s) for s in test_sent] # for the POS tag train the ner on the test set

CPU times: total: 2min 56s
Wall time: 2min 58s


In [81]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.0020339643465827964,  #was initially 0.1 each
    c2=0.028003487848126302, # 'c1': 0.2963053968677204, 'c2': 0.004195898642365605
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: total: 2min 6s
Wall time: 2min 7s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.0020339643465827964,
    c2=0.028003487848126302, max_iterations=100)

In [82]:
labels = list(crf.classes_) # get the list of all labels
labels.remove('O') # remove the ones where bio is o - not got a postag
labels # show what the labels are 

['B-PER',
 'B-ART',
 'B-CON',
 'B-LOC',
 'I-LOC',
 'I-PER',
 'I-ART',
 'B-MAT',
 'B-SPE',
 'I-MAT',
 'I-CON',
 'I-SPE']

In [83]:
#now to evaluate its success

In [84]:
#calculate the f1 score
y_pred = crf.predict(X_test) # work out and predict what is the likely token
metrics.flat_f1_score(y_test, y_pred, # work out what the likely postag will be, and give it a f1 score
                      average='weighted', labels=labels)

0.5683607193159594

In [85]:
#calculate the metrics table
# group B and I results - this isnt needed, but orders the list
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
# print(metrics.classification_report(y_test, y_pred))
print(metrics.flat_classification_report(
   y_test, y_pred, labels=sorted_labels, digits=3
))
with open ("classification report "+ fold +".txt", 'w', encoding="utf-8") as f:
    f.write(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels
))

              precision    recall  f1-score   support

       B-ART      0.642     0.427     0.513      1979
       I-ART      0.553     0.323     0.407       440
       B-CON      0.822     0.381     0.520      1576
       I-CON      0.000     0.000     0.000        49
       B-LOC      0.826     0.379     0.520       838
       I-LOC      0.319     0.236     0.271       225
       B-MAT      0.807     0.255     0.388       345
       I-MAT      0.500     0.176     0.261        17
       B-PER      0.887     0.646     0.748      1315
       I-PER      0.905     0.680     0.777      1089
       B-SPE      0.675     0.553     0.608       331
       I-SPE      0.605     0.277     0.380        94

   micro avg      0.760     0.464     0.576      8298
   macro avg      0.628     0.361     0.449      8298
weighted avg      0.758     0.464     0.568      8298



In [86]:
output = "Word  actual_tag  prediction"
for x in range(len(test_sent)):
    for i in range(len(test_sent[x])):
        testWord = test_sent[x][i][0]
        testTag = test_sent[x][i][2]
        prediction = y_pred[x][i]
        output += "\n" + testWord + "  " + testTag + "  " + prediction
        #print(testWord + "," + testTag + "," + prediction)
    output += "\n"
with open ("output "+ fold +".txt", 'w', encoding="utf-8") as f:
    f.write(output)
print(output)

Word  actual_tag  prediction
‘  [']  O
Zonnen  [Zonnen]  O
op  [op]  O
Gods  [Gods]  O
akker  [akker]  O
’  [']  O

en  [en]  O
betreft  [be][tref][t]  O
het  [het]  O
Hollandveen  [Hollandveen]  O
Laagpakket  [Laagpakket]  O
.  [.]  O

Op  [op]  O
en  [en]  O
in  [in]  O
dit  [dit]  O
veenpakket  [veen][pakket]  O
zijn  [zijn]  O
bedding  [bedding]  O
-  [-]  O
en  [en]  O
restgeulafzettingen  [rest][geulaf][zet][ing][en]  O
en  [en]  O
komafzettingen  [komaf][zet][ing][en]  O
van  [van]  O
deze  [deze]  O
rivier  [rivier]  O
aanwezig  [aanwezig]  O
.  [.]  O

De  [de]  O
stroomgordel  [stroom][gordel]  O
is  [zijn]  O
ontstaan  [ont][sta]  O
vóórdat  [vóórdat]  O
de  [de]  O
mens  [mens]  O
het  [het]  O
land  [land]  O
in  [in]  O
de  [de]  O
Late  [Late]  O
Middeleeuwen  [Middeleeuwen]  O
op  [op]  O
grootschalige  [groot][schaal][ig][e]  O
wijze  [wijze]  O
in  [in]  O
cultuur  [cultuur]  O
bracht  [breng][t]  O
.  [.]  O

Vooralsnog  [voor][alsnog]  O
wordt  [word][t]  O
uitgegaa

In [ ]:
#work out which of the transitions are most likely in descending order
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

#what are the 20 most likely transitions
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

#what are the 20 least likely transitions
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-PER  -> I-PER   5.286668
B-PER  -> I-PER   5.211908
I-ART  -> I-ART   5.068685
B-SPE  -> I-SPE   4.537802
I-MAT  -> I-MAT   4.109475
B-MAT  -> I-MAT   3.900800
B-ART  -> I-ART   3.873063
I-CON  -> I-CON   3.829557
I-LOC  -> I-LOC   3.504919
I-SPE  -> I-SPE   3.376593
O      -> O       3.312479
B-CON  -> I-CON   3.288135
B-LOC  -> I-LOC   2.829237
B-MAT  -> B-ART   1.295913
O      -> B-PER   0.993948
O      -> B-LOC   0.910349
O      -> B-ART   0.730367
I-SPE  -> B-SPE   0.689836
B-CON  -> O       0.617224
O      -> B-CON   0.596155

Top unlikely transitions:
I-MAT  -> B-MAT   -3.716338
B-PER  -> I-SPE   -3.717106
I-PER  -> I-CON   -3.907698
B-SPE  -> I-ART   -4.028105
B-ART  -> I-CON   -4.140842
I-ART  -> B-LOC   -4.315753
B-MAT  -> I-ART   -4.324852
I-LOC  -> B-PER   -4.326831
I-PER  -> I-LOC   -4.423656
I-PER  -> I-ART   -4.509269
B-PER  -> I-LOC   -4.621955
B-LOC  -> I-ART   -5.002381
O      -> I-MAT   -6.308835
B-PER  -> I-CON   -6.895113
O      -> I-SPE  

In [ ]:
#What aspects of the terms make it likely to be that tag
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

#what are the 30 most likely aspects
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

#what 30 aspects make it least likely to be that term
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
10.366779 B-LOC    +2:word.lower():hei141
10.133368 B-MAT    word.lower():koperlegering
9.954262 B-SPE    word.lower():runderen
9.798606 B-LOC    +2:word.lower():burg.
9.515946 B-CON    word.lower():paalsporen
9.288272 B-CON    word.lower():paalspoor
9.092768 B-CON    word.lower():paalkuilen
8.918348 B-SPE    word.lower():vissen
8.862185 B-CON    word.lower():vuursteenmijnen
8.810996 B-SPE    word.lower():bunzing
8.793311 B-LOC    -1:word.lower():zoom
8.733163 B-MAT    word.lower():geglazuurde
8.730110 B-CON    word.lower():vuursteenmijn
8.719548 B-CON    word.lower():veekraal
8.641187 B-ART    word.lower():afvaldepositie
8.442832 B-ART    word.lower():gatenpotten
8.365664 B-PER    word.lower():prehistorie
8.335927 B-CON    word.lower():grafveld
8.288101 B-SPE    word.lower():rapen
8.277285 B-ART    word.lower():molaar
8.186384 B-PER    word.lower():bronstijdboeren
8.088271 B-LOC    word.lower():geldermalsen-stationslocatie
8.052235 B-SPE    word.lower():paarden
7.993515 

In [ ]:
## dont run this takes too long

In [ ]:
%%time 
# this is to work out the best parameters for the testing. not needed yet but can increase the results by .1 
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [ ]:
# crf = rs.best_estimator_  # shows that the best params are 
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
_x = [s.parameters['c1'] for s in rs.grid_scores_]
_y = [s.parameters['c2'] for s in rs.grid_scores_]
_c = [s.mean_validation_score for s in rs.grid_scores_]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))